接續方法2剩下7個位點 進行3個為一組組合 使用5種模型預測

In [2]:
import pandas as pd
import xgboost as xgb

In [ ]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_0.35.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_0.35.csv')

normalized_train = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

method2_gene=pd.read_csv("../result/overlap_select_gene_7.csv")
normalized_train_c= normalized_train[normalized_train['Unnamed: 0'].isin(method2_gene['ID'])]
normalized_train_c

In [ ]:
X_train = normalized_train_c.iloc[:, 1::2]

x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

y = [ (0 if i < 47 else 1)  for i in range(445)]

In [15]:
from sklearn.metrics import roc_auc_score, f1_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
target_labels = normalized_train_c['Unnamed: 0']
# 三個一組
combinations_list = list(combinations(target_labels, 3))

result=[]
# 定義模型參數
params = {
    'objective': 'binary:logistic',  # 二分類問題
    'eval_metric': 'logloss',  # 評估指標
    'max_depth': 3,
    'learning_rate': 1
}
models = {
    'XGBoost': xgb.XGBClassifier(**params),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='linear'),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():

    # 所有可能的組合
    for combination in combinations_list:
        #print(combination)
        d1_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[0]]
        d2_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[1]]
        d3_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[2]]
    
        d1 = d1_values.iloc[:, 1::2]
        d2 = d2_values.iloc[:, 1::2]
        d3 = d3_values.iloc[:, 1::2]

        d1 = d1.values.flatten().tolist()
        d2 = d2.values.flatten().tolist()
        d3 = d3.values.flatten().tolist()

        x = []

        for i in range(445):
            x.append([d1[i],d2[i],d3[i]])
            
        # 將數據轉換為DMatrix格式
        dmatrix = xgb.DMatrix(x, label=y)

        # 進行交叉驗證，這裡使用 5-fold 交叉驗證
        cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
        # 計算平均準確度
        average_accuracy = cv_scores.mean()

        # 獲取模型預測結果
        y_pred = cross_val_predict(model, x, y, cv=5)
        
        # 計算AUC
        auc_score = roc_auc_score(y, y_pred)
        
        # 計算F1 score
        f1 = f1_score(y, y_pred)
        GO_1=int(method2_gene.loc[method2_gene['ID'] == combination[0]]["GO"])
        GO_2=int(method2_gene.loc[method2_gene['ID'] == combination[1]]["GO"])
        GO_3=int(method2_gene.loc[method2_gene['ID'] == combination[2]]["GO"])
        if (GO_1!=GO_2 & GO_2!=GO_3 & GO_1!=GO_3):
            print(GO_1,GO_2,GO_3)
            if (GO_1==1 and GO_2==4 and GO_3==3):
                result.append([model_name, combination[0], combination[2], combination[1], average_accuracy, auc_score, f1])
            elif (GO_1==1 and GO_2==3 and GO_3==4):
                result.append([model_name, combination[0], combination[1], combination[2], average_accuracy, auc_score, f1])
            elif (GO_1==4 and GO_2==1 and GO_3==3):
                result.append([model_name, combination[1], combination[2], combination[0], average_accuracy, auc_score, f1])
            elif (GO_1==4 and GO_2==3 and GO_3==1):
                result.append([model_name, combination[2], combination[1], combination[0], average_accuracy, auc_score, f1])
            elif (GO_1==3 and GO_2==1 and GO_3==4):
                result.append([model_name, combination[1], combination[0], combination[2], average_accuracy, auc_score, f1])
            elif (GO_1==3 and GO_2==4 and GO_3==1):
                result.append([model_name, combination[2], combination[0], combination[1], average_accuracy, auc_score, f1])
            print(result)  
result


In [ ]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'ID3', 'average_accuracy','AUC','f1_score'])
df.to_csv("../result/ml_combination_method2.csv", index=False)

挑出3個為一組最佳的組合: MIR124-2、ZSCAN18、KCNE3 繪製normal/tumor間的關係

In [ ]:
d1_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg07792478']
d2_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg04574090']
d3_values = normalized_train.loc[normalized_train['Unnamed: 0'] == 'cg14231297']


print(d1_values)

d1 = d1_values.iloc[:, 1::2]
d2 = d2_values.iloc[:, 1::2]
d3 = d3_values.iloc[:, 1::2]

d1 = d1.values.flatten().tolist()
d2 = d2.values.flatten().tolist()
d3 = d3.values.flatten().tolist()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

def plot_3d_scatter(elev, azim):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(d1[:47], d2[:47], d3[:47], marker='o', color='blue', label='normal')
    ax.scatter(d1[47:], d2[47:], d3[47:], marker='x', color='red', label='tumor')

    ax.set_title('3D Scatter Plot')
    ax.set_xlabel('X-cg07792478')
    ax.set_ylabel('Y-cg04574090')
    ax.set_zlabel('Z-cg14231297')

    ax.legend()

    ax.view_init(elev=elev, azim=azim)
    
    plt.show()

interact(plot_3d_scatter, elev=(0, 90, 5), azim=(0, 360, 5))


接續方法1剩下40個位點進行GO分4群後實作RFECV總共剩下30個 於每群挑一個代表的位點 組合並預測

In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_0.35.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_0.35.csv')

normalized_train = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

cluster=pd.read_csv("../result/RFE_method1/RFECV/cluster.csv")
normalized_train_c= normalized_train[normalized_train['Unnamed: 0'].isin(cluster['ID'])]
normalized_train_c

In [ ]:
X_train = normalized_train_c.iloc[:, 1::2]

x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

y = [ (0 if i < 47 else 1)  for i in range(445)]

In [ ]:
def is_unique(lst):
    return len(lst) == len(set(lst))

In [ ]:
target_labels = normalized_train_c['Unnamed: 0']
# 四個一組
combinations_list = list(combinations(target_labels, 4))

result=[]
# 定義模型參數
params = {
    'objective': 'binary:logistic',  # 二分類問題
    'eval_metric': 'logloss',  # 評估指標
    'max_depth': 3,
    'learning_rate': 1
}
models = {
    'XGBoost': xgb.XGBClassifier(**params),
    'RandomForest': RandomForestClassifier(n_estimators=100, 
                                            max_depth=10, 
                                            min_samples_split=5, 
                                            min_samples_leaf=2,
                                            min_impurity_decrease=0.0,
                                            random_state=42),
    'SVM': SVC(kernel='linear'),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():

    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(cluster.loc[cluster['ID'] == combination[0]]["GO"])
        GO_2=int(cluster.loc[cluster['ID'] == combination[1]]["GO"])
        GO_3=int(cluster.loc[cluster['ID'] == combination[2]]["GO"])
        GO_4=int(cluster.loc[cluster['ID'] == combination[3]]["GO"])

        lst=[GO_1,GO_2,GO_3,GO_4]

        if(is_unique(lst)):
            d1_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[0]]
            d2_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[1]]
            d3_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[2]]
            d4_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[3]]
        
            d1 = d1_values.iloc[:, 1::2]
            d2 = d2_values.iloc[:, 1::2]
            d3 = d3_values.iloc[:, 1::2]
            d4 = d4_values.iloc[:, 1::2]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            d3 = d3.values.flatten().tolist()
            d4 = d4.values.flatten().tolist()

            x = []

            for i in range(445):
                x.append([d1[i],d2[i],d3[i],d4[i]])
            

            dmatrix = xgb.DMatrix(x, label=y)

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1])
            
# result


In [ ]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'ID3','ID4', 'average_accuracy','AUC','f1_score'])
df.to_csv("../result/RFE_method1/RFECV/ml_combination_method1.csv", index=False)

In [ ]:
method1_combination=pd.read_csv("../result/RFE_method1/RFECV/ml_combination_method1.csv")

for i in range(0,len(method1_combination)-4,5):
    method1_combination['AVG_accuracy'][i]=(method1_combination['average_accuracy'][i:i+5]).mean()
    method1_combination['AVG_AUC'][i]=(method1_combination['AUC'][i:i+5]).mean()
    method1_combination['AVG_f1_score'][i]=(method1_combination['f1_score'][i:i+5]).mean()

In [ ]:
method1_combination.to_csv("../result/RFE_method1/RFECV/ml_combination_method1.csv", index=False)

挑選出4個為一組最佳組合:RGS22、SLC6A3、GHSR、ZSCAN18(和方法2相同)

#### GSE243529 組合預測

In [1]:
#GSE243529
import pandas as pd
# liquid_train=pd.read_csv("../../乳癌液態/GSE243529/train/all_beta_normalized_train.csv")
liquid_test=pd.read_csv("../../乳癌液態/GSE243529/test/all_beta_normalized_test.csv")

In [3]:
method_gene=pd.DataFrame(columns=['ID','Cluster'])
method_gene['ID'] =['cg02676175','cg13379236','cg19538614','cg15744637','cg01305745']
method_gene['Cluster']=[2,3,3,3,3]
 

In [3]:
# liquid_train.set_index('Unnamed: 0', inplace=True)#df1
# liquid_test.set_index('Unnamed: 0', inplace=True)#df2

# liquid = pd.merge(liquid_train, liquid_test, left_index=True, right_index=True, suffixes=('_df1', '_df2'))
# liquid.reset_index(inplace=True)
liquid = liquid_test[liquid_test['Unnamed: 0'].isin(method_gene['ID'])]
liquid = liquid.iloc[:,::2]
liquid

,Unnamed: 0,2,4,6,8,10,12,14,16,18,...,506,508,510,512,514,516,518,520,522,524
496676,cg13379236,0.528638,0.652024,0.623877,0.600521,0.651696,0.698779,0.626863,0.667152,0.694145,...,0.727405,0.537415,0.591056,0.557628,0.662244,0.623344,0.552558,0.596685,0.677702,0.625546
625322,cg19538614,0.203102,0.239800,0.215799,0.220656,0.247154,0.262032,0.239807,0.217848,0.253634,...,0.228220,0.241566,0.226783,0.255958,0.210242,0.244206,0.276625,0.235694,0.304671,0.264651
643514,cg02676175,0.648279,0.352880,0.386686,0.538006,0.562625,0.711593,0.622941,0.670064,0.692824,...,0.337300,0.409056,0.513829,0.365483,0.455773,0.543678,0.476182,0.182500,0.562093,0.499381
651559,cg15744637,0.124614,0.123013,0.104267,0.110797,0.117954,0.109890,0.114580,0.122934,0.140475,...,0.114481,0.125738,0.130422,0.158126,0.137657,0.150671,0.208612,0.131290,0.135042,0.109045
704780,cg01305745,0.661353,0.709735,0.671287,0.661975,0.758612,0.764028,0.697917,0.694349,0.748629,...,0.729499,0.578589,0.679820,0.679149,0.642883,0.707826,0.733224,0.676659,0.722411,0.689949


In [10]:
X_train = liquid.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(261)]
# y = [ (1 if i < 128 else 0)  for i in range(262)] # 128 tumor 134 normal
y = [(1 if i < 134 else 0)  for i in range(261)] # 134 tumor 127 normal
# y=liquid_label_train+liquid_label_test

In [8]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score,confusion_matrix,accuracy_score,precision_score
from itertools import combinations
from sklearn.model_selection import cross_val_score,cross_val_predict,train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
import pandas as pd
import numpy as np
from itertools import combinations


In [9]:
def is_unique(lst):
    return len(lst) == len(set(lst))

In [11]:
param_grids = {
    "SVM": {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    "Logistic Regression": {
        'C': [0.1, 1, 10],
        'solver': ['liblinear']
    },
    "Decision Tree": {
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    },
    "RandomForest": {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    },
    "XGBoost": {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2]
    }
}

In [14]:
#cv=5
target_labels = method_gene['ID']
# 兩個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['Unnamed: 0']==combination[0]]
            d2_values = liquid.loc[liquid['Unnamed: 0']==combination[1]]
            # d3_values = liquid.loc[liquid['Unnamed: 0']==combination[2]]
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()

            x = []

            for i in range(261):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

In [15]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y, y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.58      0.84      0.69       127
           1       0.74      0.43      0.55       134

    accuracy                           0.63       261
   macro avg       0.66      0.64      0.62       261
weighted avg       0.67      0.63      0.62       261



In [16]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_GSE243529.csv", index=False)

#### GSE89093

In [27]:
import pandas as pd
liquid=pd.read_csv("../../乳癌液態/GSE89093/GSE89093_all_beta_normalized.csv")
liquid_label= pd.read_csv("../../乳癌液態/GSE89093/GSE89093_phenotype.csv")
liquid_label=pd.DataFrame(liquid_label["cancer_status"])
liquid= liquid[liquid['ID_REF'].isin(method_gene['ID'])]
liquid

,ID_REF,0,1,2,3,4,5,6,7,8,...,82,83,84,85,86,87,88,89,90,91
25471,cg01305745,0.638166,0.680956,0.681443,0.621832,0.672060,0.656032,0.652598,0.715919,0.739194,...,0.660671,0.680028,0.555772,0.612494,0.657143,0.660234,0.653082,0.640170,0.731687,0.647158
50982,cg02676175,0.549307,0.437833,0.494183,0.720193,0.787223,0.421922,0.409400,0.647108,0.552143,...,0.491332,0.622798,0.499606,0.536625,0.437171,0.162358,0.484223,0.360982,0.354604,0.412771
230122,cg13379236,0.694362,0.637849,0.632367,0.651982,0.719588,0.718063,0.713096,0.706998,0.782798,...,0.775504,0.703100,0.742238,0.777616,0.690528,0.670476,0.693118,0.763106,0.676052,0.685293
271498,cg15744637,0.028924,0.047274,0.044786,0.038761,0.039348,0.041374,0.044651,0.043858,0.022129,...,0.034486,0.026328,0.029617,0.028564,0.040906,0.039448,0.034916,0.045758,0.040178,0.038868
329121,cg19538614,0.073263,0.175869,0.157871,0.181264,0.141273,0.134463,0.131353,0.275797,0.117453,...,0.168827,0.128677,0.100663,0.089679,0.143254,0.143560,0.140922,0.119762,0.173738,0.138592


In [28]:
X_train = liquid.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(92)]
y = [0 if liquid_label.iloc[i,0] == 'healthy' else 1 for i in range(liquid_label.shape[0])]

In [29]:
def is_unique(lst):
    return len(lst) == len(set(lst))

In [30]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['ID_REF']==combination[0]]
            d2_values = liquid.loc[liquid['ID_REF']==combination[1]]
            # d3_values = liquid.loc[liquid['ID_REF']==combination[2]]
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()

            x = []
            for i in range(92):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

In [31]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y, y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.44      0.41      0.43        46
           1       0.45      0.48      0.46        46

    accuracy                           0.45        92
   macro avg       0.45      0.45      0.45        92
weighted avg       0.45      0.45      0.45        92



In [32]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_GSE89093.csv", index=False)

#### GSE148663烏拉圭

In [17]:
import pandas as pd
liquid=pd.read_csv("../../乳癌液態/GSE148663烏拉圭/all_beta_normalized_breast_liquid_oversample_random.csv")
liquid= liquid[liquid['Unnamed: 0'].isin(method_gene['ID'])]

liquid

,Unnamed: 0,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,1.12,1.13,1.14,1.15,1.16,1.17,1.18,1.19,1.20,1.21
65410,cg15744637,0.092601,0.078883,0.070746,0.064469,0.150053,0.075991,0.064469,0.078883,0.070746,...,0.074580,0.080159,0.083957,0.071556,0.083098,0.069817,0.073028,0.073735,0.091964,0.061401
84664,cg01305745,0.652978,0.742951,0.749625,0.770792,0.612471,0.751017,0.770792,0.742951,0.749625,...,0.593302,0.646993,0.724909,0.713370,0.725306,0.720394,0.734664,0.737786,0.714340,0.753672
90677,cg02676175,0.361315,0.521305,0.616833,0.533898,0.638511,0.720535,0.533898,0.521305,0.616833,...,0.732635,0.590756,0.776856,0.657576,0.331018,0.523803,0.373197,0.431849,0.492970,0.627320
104368,cg19538614,0.191818,0.147849,0.398092,0.194692,0.190211,0.224114,0.194692,0.147849,0.398092,...,0.273539,0.222596,0.227264,0.188412,0.209681,0.163641,0.228998,0.167897,0.192970,0.182801
189019,cg13379236,0.806629,0.802891,0.844101,0.846237,0.803951,0.844067,0.846237,0.802891,0.844101,...,0.792853,0.792865,0.819751,0.807945,0.829341,0.800345,0.797902,0.828199,0.814068,0.787952


In [18]:
X_train = liquid.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(44)]
y = [ (0 if i < 22 else 1)  for i in range(44)]

In [19]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['Unnamed: 0']==combination[0]]
            d2_values = liquid.loc[liquid['Unnamed: 0']==combination[1]]
            # d3_values = liquid.loc[liquid['Unnamed: 0']==combination[2]]
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()

            x = []

            for i in range(44):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

In [20]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y,y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.70      0.86      0.78        22
           1       0.82      0.64      0.72        22

    accuracy                           0.75        44
   macro avg       0.76      0.75      0.75        44
weighted avg       0.76      0.75      0.75        44



In [21]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_GSE148663.csv", index=False)

#### GSE133918母乳

In [22]:
import pandas as pd
liquid=pd.read_csv("../../乳癌液態/GSE133918_breast_milk/all_beta_normalized_milk.csv")
liquid= liquid[liquid['Unnamed: 0'].isin(method_gene['ID'])]

liquid

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
64351,cg15744637,0.063244,0.063244,0.042315,0.042315,0.044898,0.044898,0.059296,0.059296,0.041902,...,0.073806,0.073806,0.055999,0.055999,0.058153,0.058153,0.062709,0.062709,0.069802,0.069802
83317,cg01305745,0.874856,0.874856,0.790932,0.790932,0.810628,0.810628,0.811630,0.811630,0.698743,...,0.808429,0.808429,0.770529,0.770529,0.836257,0.836257,0.727968,0.727968,0.799523,0.799523
89259,cg02676175,0.140642,0.140642,0.145255,0.145255,0.139482,0.139482,0.131759,0.131759,0.389052,...,0.191928,0.191928,0.305602,0.305602,0.093809,0.093809,0.326969,0.326969,0.168103,0.168103
102782,cg19538614,0.080142,0.080142,0.080357,0.080357,0.095419,0.095419,0.080736,0.080736,0.119978,...,0.150418,0.150418,0.097530,0.097530,0.082395,0.082395,0.099932,0.099932,0.077812,0.077812
186533,cg13379236,0.739176,0.739176,0.590453,0.590453,0.740707,0.740707,0.653871,0.653871,0.776010,...,0.650710,0.650710,0.678955,0.678955,0.508684,0.508684,0.702186,0.702186,0.665681,0.665681


In [23]:
X_train = liquid.iloc[:, 1::2]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(80)]
y = [ (0 if i < 64 else 1)  for i in range(80)]

In [24]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])
        

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = liquid.loc[liquid['Unnamed: 0']==combination[0]]
            d2_values = liquid.loc[liquid['Unnamed: 0']==combination[1]]
            # d3_values = liquid.loc[liquid['Unnamed: 0']==combination[2]]
           
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]
           

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()
           

            x = []

            for i in range(80):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1471: Un

In [25]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y,y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.85      0.80      0.82        64
           1       0.35      0.44      0.39        16

    accuracy                           0.73        80
   macro avg       0.60      0.62      0.61        80
weighted avg       0.75      0.72      0.74        80



In [26]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_GSE133918.csv", index=False)

#### 850K組織

In [33]:
import pandas as pd
tissue=pd.read_csv("../../乳癌資料集850/all_beta_normalized_850(GSE237036_RAW).csv")
tissue= tissue[tissue['Unnamed: 0'].isin(method_gene['ID'])]
tissue

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
499535,cg13379236,0.763281,0.763281,0.818032,0.818032,0.760685,0.760685,0.718002,0.718002,0.744761,...,0.734326,0.734326,0.766877,0.766877,0.752042,0.752042,0.723260,0.723260,0.721929,0.721929
628048,cg19538614,0.276054,0.276054,0.318521,0.318521,0.270917,0.270917,0.183618,0.183618,0.202970,...,0.293218,0.293218,0.293265,0.293265,0.259124,0.259124,0.213234,0.213234,0.382358,0.382358
643600,cg02676175,0.533065,0.533065,0.524492,0.524492,0.392884,0.392884,0.599335,0.599335,0.831313,...,0.573791,0.573791,0.535702,0.535702,0.726381,0.726381,0.667751,0.667751,0.684275,0.684275
650493,cg15744637,0.053368,0.053368,0.049018,0.049018,0.040947,0.040947,0.043140,0.043140,0.035678,...,0.063712,0.063712,0.048973,0.048973,0.051649,0.051649,0.058925,0.058925,0.054024,0.054024
696210,cg01305745,0.799745,0.799745,0.824651,0.824651,0.791297,0.791297,0.795163,0.795163,0.799044,...,0.768198,0.768198,0.709310,0.709310,0.725467,0.725467,0.733396,0.733396,0.737386,0.737386


In [34]:
X_train = tissue.iloc[:, 1::2]
y = [ (1 if i < 50 else 0)  for i in range(80)]

In [35]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])
        

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = tissue.loc[tissue['Unnamed: 0']==combination[0]]
            d2_values = tissue.loc[tissue['Unnamed: 0']==combination[1]]
            # d3_values = tissue.loc[tissue['Unnamed: 0']==combination[2]]
        
        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]
           

            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()
          
            x = []

            for i in range(80):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

In [36]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y,y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.33      0.27      0.30        30
           1       0.61      0.68      0.64        50

    accuracy                           0.53        80
   macro avg       0.47      0.47      0.47        80
weighted avg       0.50      0.53      0.51        80



In [37]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_850K.csv", index=False)

#### 450K組織

In [4]:
tissue_train = pd.read_csv('../../all_beta_normalized_train.csv')
tissue_test = pd.read_csv('../../all_beta_normalized_test.csv')

In [5]:
tissue_train.set_index('Unnamed: 0', inplace=True)#df1
tissue_test.set_index('Unnamed: 0', inplace=True)#df2

tissue = pd.merge(tissue_train, tissue_test, left_index=True, right_index=True, suffixes=('_df1', '_df2'))
tissue.reset_index(inplace=True)
tissue = tissue[tissue['Unnamed: 0'].isin(method_gene['ID'])]
tissue = tissue.iloc[:,::2]
tissue

,Unnamed: 0,2_df1,4_df1,6_df1,8_df1,10_df1,12_df1,14_df1,16_df1,18_df1,...,882_df2,884_df2,886_df2,888_df2,890_df2,892,894,896,898,900
56425,cg15744637,0.047868,0.031748,0.043978,0.054314,0.034021,0.057449,0.037729,0.049467,0.038572,...,0.033289,0.034189,0.011312,0.022018,0.019680,0.022243,0.039181,0.032738,0.031085,0.067296
72978,cg01305745,0.695101,0.673430,0.662219,0.733781,0.798592,0.796947,0.792315,0.780187,0.812400,...,0.850521,0.913364,0.106866,0.886185,0.567833,0.872566,0.884640,0.901377,0.850846,0.368562
78297,cg02676175,0.262729,0.090647,0.188114,0.098290,0.123440,0.106426,0.133582,0.190340,0.096213,...,0.155620,0.096188,0.090891,0.077723,0.187307,0.180139,0.092540,0.099415,0.091777,0.342642
90467,cg19538614,0.069397,0.046278,0.103853,0.080952,0.083748,0.061636,0.075952,0.072473,0.098416,...,0.077161,0.059008,0.166385,0.382460,0.473965,0.184197,0.065280,0.081902,0.071313,0.309062
159536,cg13379236,0.856664,0.794824,0.877258,0.817793,0.856344,0.863313,0.802324,0.818501,0.816200,...,0.562665,0.281121,0.721915,0.926260,0.890527,0.615370,0.485040,0.505763,0.729556,0.849225


In [6]:
X_train = tissue.iloc[:, 1::]
x = [X_train.iloc[:, i].values.flatten().tolist() for i in range(895)]
tissue_label_train = [ (0 if i < 47 else 1)  for i in range(445)] # 398 tumor/47 normal =445
tissue_label_test = [(0 if i < 50 else 1)  for i in range(450)] # 400 tumor/50 normal =450
y=tissue_label_train+tissue_label_test

In [12]:
#cv=5
target_labels = method_gene['ID']
# 三個一組
combinations_list = list(combinations(target_labels, 2))
result=[]

models = {
    'XGBoost': xgb.XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier()
}

for model_name, model in models.items():
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    # 所有可能的組合
    for combination in combinations_list:

        GO_1=int(method_gene.loc[method_gene['ID'] == combination[0]]["Cluster"])
        GO_2=int(method_gene.loc[method_gene['ID'] == combination[1]]["Cluster"])
        # GO_3=int(method_gene.loc[method_gene['ID'] == combination[2]]["Cluster"])
   

        lst=[GO_1,GO_2]

        if(is_unique(lst)):
            d1_values = tissue.loc[tissue['Unnamed: 0']==combination[0]]
            d2_values = tissue.loc[tissue['Unnamed: 0']==combination[1]]
            # d3_values = tissue.loc[tissue['Unnamed: 0']==combination[2]]

        
            d1 = d1_values.iloc[:, 1::]
            d2 = d2_values.iloc[:, 1::]
            # d3 = d3_values.iloc[:, 1::]


            d1 = d1.values.flatten().tolist()
            d2 = d2.values.flatten().tolist()
            # d3 = d3.values.flatten().tolist()


            x = []
            for i in range(895):
                x.append([d1[i],d2[i]])

            grid_search.fit(x, y)
            model = grid_search.best_estimator_

            cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')
            average_accuracy = cv_scores.mean()

            # 獲取模型預測結果
            y_pred = cross_val_predict(model, x, y, cv=5)
            
            # 預測
            tn, fp, fn, tp = confusion_matrix(y, y_pred).ravel()
            sensitivity = tp / (tp + fn)
            specificity = tn / (tn + fp)
            precision = precision_score(y, y_pred)
            auc_score = roc_auc_score(y, y_pred)
            f1 = f1_score(y, y_pred)

            sorted_combination = [x for _, x in sorted(zip(lst, combination))]
            result.append([model_name] + sorted_combination + [average_accuracy, auc_score, f1, sensitivity,specificity,precision])
            
# result

In [13]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(y, y_pred,labels=np.unique(y_pred)))

              precision    recall  f1-score   support

           0       0.45      0.55      0.50        97
           1       0.94      0.92      0.93       798

    accuracy                           0.88       895
   macro avg       0.70      0.73      0.71       895
weighted avg       0.89      0.88      0.88       895



In [14]:
df=pd.DataFrame(result, columns=['Model','ID1', 'ID2', 'average_accuracy','AUC','f1_score','sensitivity','specificity','precision'])
df.to_csv("../result/ml_combination_450K.csv", index=False)